<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/2025_06_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [ ]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [ ]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [ ]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [ ]:
# ✅ 라이브러리 로드
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
try:
    train = pd.read_csv(get_path("train.csv"))
    test = pd.read_csv(get_path("test.csv"))
    submission = pd.read_csv(get_path("sample_submission.csv"))
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")

# ✅ 타겟/피처 분리
y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

# ✅ log1p 정규화 함수
def apply_log1p(df):
    df = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        if (df[col] <= 0).any():
            df[col] = np.log1p(df[col] - df[col].min() + 1)
        else:
            df[col] = np.log1p(df[col])
    return df

# ✅ 전처리 + log1p 적용
X, encoders = preprocess(X)
X = apply_log1p(X)

X_test, _ = preprocess(X_test, encoders)
X_test = apply_log1p(X_test)

# ✅ 기본 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)

# ✅ 메타 모델 정의
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# ✅ 스태킹 모델 정의
base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ OOF 예측 및 Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 전체 데이터로 학습 & 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]

# ✅ 제출 파일 생성
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_log1p.csv", index=False)
print("✅ 제출 파일 저장 완료: submission_stacking_log1p.csv")


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002501 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002329 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002345 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Threshold: 0.17, Best F1 Score: 0.48573
[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# ✅ 전략 2: F2 Score 기반 Threshold 최적화 포함 전체 코드 (Google Colab 호환)

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import precision_score, recall_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수 (Colab 기준)
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 기본 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# ✅ 스태킹 모델 정의
base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ 직접 OOF 예측 수행
oof_preds = np.zeros(len(X))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, valid_idx in skf.split(X, y):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    stack_model.fit(X_train, y_train)
    oof_preds[valid_idx] = stack_model.predict_proba(X_valid)[:, 1]

# ✅ F2 Score 기반 Threshold 최적화 함수
def fbeta_score_custom(y_true, y_prob, beta=2.0):
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    for t in thresholds:
        y_pred = (y_prob > t).astype(int)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        beta_sq = beta ** 2
        fbeta = (1 + beta_sq) * (precision * recall) / (beta_sq * precision + recall + 1e-8)
        scores.append(fbeta)
    best_idx = np.argmax(scores)
    return thresholds[best_idx], scores[best_idx]

# ✅ Threshold 선택
best_thresh, best_fbeta = fbeta_score_custom(y, oof_preds, beta=2.0)
print(f"📌 Best Threshold (F2): {best_thresh:.2f}, Best F2 Score: {best_fbeta:.5f}")

# ✅ 전체 데이터로 학습 후 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_f2.csv", index=False)
print("✅ 제출 파일 저장 완료: submission_stacking_f2.csv")


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002449 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_di

📌 Best Threshold (F2): 0.25, Best F2 Score: 0.47059


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002932 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# ✅ 라이브러리 로드
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ 타겟/피처 분리
y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

# ✅ log1p 정규화 함수
def apply_log1p(df):
    df = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        if (df[col] <= 0).any():
            df[col] = np.log1p(df[col] - df[col].min() + 1)
        else:
            df[col] = np.log1p(df[col])
    return df

# ✅ 전처리 + log1p 적용
X, encoders = preprocess(X)
X = apply_log1p(X)
X_test, _ = preprocess(X_test, encoders)
X_test = apply_log1p(X_test)

# ✅ 기본 모델 정의 (Class Weight 조정)
model_lgbm = LGBMClassifier(
    learning_rate=0.05,
    max_depth=6,
    num_leaves=31,
    class_weight='balanced',
    random_state=42
)

model_xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=(y == 0).sum() / (y == 1).sum(),  # XGBoost에서는 직접 비율 설정
    random_state=42
)

model_cat = CatBoostClassifier(
    verbose=0,
    class_weights=[(y == 0).sum() / len(y), (y == 1).sum() / len(y)],  # CatBoost은 리스트 형태
    random_state=42
)

# ✅ 메타 모델 정의
meta_model = LGBMClassifier(
    learning_rate=0.05,
    max_depth=3,
    num_leaves=15,
    random_state=42
)

# ✅ 스태킹 모델 정의
base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ OOF 예측 및 Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 전체 데이터로 학습 & 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]

# ✅ 제출 파일 생성
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_class_weight.csv", index=False)
print("✅ 제출 파일 저장 완료: submission_class_weight.csv")


In [ ]:
# 추가전략 1
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    # Colab에서 파일을 업로드했거나 드라이브를 마운트했다면 해당 경로로 변경해야 합니다.
    # 예시: 구글 드라이브 마운트 후 'My Drive/data/' 폴더에 데이터가 있다면
    # return "/content/drive/MyDrive/data/" + filename
    # 현재 가정: Colab에 직접 파일을 업로드했고, 해당 파일들이 /content/open_1/ 에 있다고 가정
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
try:
    train_df = pd.read_csv(get_path("train.csv"))
    test_df = pd.read_csv(get_path("test.csv"))
    submission_df = pd.read_csv(get_path("sample_submission.csv"))
except FileNotFoundError: # try 블록 안에 있어야 합니다.
    print("데이터셋 경로를 찾을 수 없습니다. Colab에 파일을 올바르게 업로드했는지 확인하거나 get_path 함수를 수정해주세요.")
    print("예: train.csv, test.csv, sample_submission.csv 파일이 /content/open_1/ 경로에 있는지 확인하세요.")
    exit()

y = train_df["Cancer"]
X_raw = train_df.drop(columns=["ID", "Cancer"]) # 원본 X (전처리 전)
X_test_raw = test_df.drop(columns=["ID"]) # 원본 X_test (전처리 전)

# ✅ 전처리 전 원본 X_raw에서 범주형 컬럼 리스트 미리 추출 (전역에서 사용)
original_categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()


# ✅ 전처리 함수
def preprocess(df, encoders=None, is_train=True): # original_X_raw_cols 인자 제거
    df_processed = df.copy()
    encoders = encoders or {}

    for col in df_processed.columns:
        if col in original_categorical_cols: # 원본 범주형 컬럼에 대해서만 Label Encoding
            df_processed[col] = df_processed[col].astype(str)
            if is_train and col not in encoders:
                encoders[col] = LabelEncoder().fit(df_processed[col])
            # test 데이터에 train에 없는 새로운 범주가 나타날 경우를 대비
            df_processed[col] = df_processed[col].map(lambda s: encoders[col].transform([s])[0] if s in encoders[col].classes_ else -1)
        elif df_processed[col].dtype == "object": # Label Encoding 대상이 아닌데 object 타입인 경우 (새로운 FE 컬럼 등)
            pass
        else: # 수치형 컬럼
            imputer_key = f"imputer_{col}"
            if is_train:
                encoders[imputer_key] = SimpleImputer(strategy="mean").fit(df_processed[[col]])
            if imputer_key in encoders: # train에서 학습된 imputer가 있는 경우 사용
                df_processed[col] = encoders[imputer_key].transform(df_processed[[col]])
    return df_processed, encoders

# 데이터 전처리 적용
X_processed, encoders_fitted = preprocess(X_raw, is_train=True)
X_test_processed, _ = preprocess(X_test_raw, encoders=encoders_fitted, is_train=False) # encoders_fitted 사용


# ✅ Feature Interaction 파생변수 추가 함수
def add_interactions(df, cols):
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    inter_feats_array = poly.fit_transform(df[cols])

    feature_names = poly.get_feature_names_out(cols)

    inter_df = pd.DataFrame(inter_feats_array, columns=feature_names, index=df.index)

    interaction_only_df = inter_df.iloc[:, len(cols):] # 원본 컬럼 제외

    return pd.concat([df, interaction_only_df], axis=1)

# Feature Interaction 적용
interaction_cols = X_processed.select_dtypes(include=np.number).columns.tolist()
X_inter = add_interactions(X_processed, interaction_cols)
X_test_inter = add_interactions(X_test_processed, interaction_cols)

# CatBoost를 위한 범주형 피처 인덱스 업데이트 (상호작용 피처 추가 후)
# X_inter (모든 전처리 및 상호작용 피처가 추가된 최종 DataFrame)에서
# original_categorical_cols에 해당하는 컬럼들의 현재 인덱스를 다시 찾습니다.
cat_features_indices_for_catboost_final = [
    X_inter.columns.get_loc(col) for col in original_categorical_cols if col in X_inter.columns
]


# ✅ 모델 정의
# CatBoost에 cat_features 명시
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42, n_jobs=-1, verbose=-1)
# Colab 무료 티어는 GPU가 항상 가용하지 않으므로 일단 CPU로 설정. 필요시 'GPU'로 변경
model_cat = CatBoostClassifier(verbose=0, random_state=42, cat_features=cat_features_indices_for_catboost_final, task_type='CPU')
meta_model = LGBMClassifier(learning_rate=0.03, max_depth=4, num_leaves=16, random_state=42, n_jobs=-1, verbose=-1)

# ✅ 스태킹 모델 정의
stack_model = StackingClassifier(
    estimators=[
        ('lgbm', model_lgbm),
        ('cat', model_cat)
    ],
    final_estimator=meta_model,
    passthrough=True, # 기본 모델의 예측 외에 원본 특성도 메타 모델에 전달
    cv=5,
    stack_method='predict_proba', # 기본 모델의 확률 예측을 메타 모델에 전달
    n_jobs=-1 # 병렬 처리
)

# ✅ OOF 예측 및 Threshold 최적화 (F1 Score 기반 최적화 전략)
print("Stacking 모델 OOF 예측 시작...")
oof_preds = cross_val_predict(stack_model, X_inter, y, method="predict_proba", cv=5)[:, 1]

thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"✅ Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 학습 및 테스트 예측
print("최종 Stacking 모델 학습 시작...")
stack_model.fit(X_inter, y)
print("최종 Stacking 모델 예측 시작...")
test_preds = stack_model.predict_proba(X_test_inter)[:, 1]

submission_df["Cancer"] = (test_preds > best_thresh).astype(int)
submission_df.to_csv("submission_stack_inter_threshopt_colab.csv", index=False) # Colab용 파일명 변경
print("✅ 저장 완료: submission_stack_inter_threshopt_colab.csv")

Stacking 모델 OOF 예측 시작...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ Best Threshold: 0.24, Best F1 Score: 0.48672
최종 Stacking 모델 학습 시작...
최종 Stacking 모델 예측 시작...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ 저장 완료: submission_stack_inter_threshopt_colab.csv


In [ ]:
from google.colab import files
files.download("submission_stack_inter_threshopt_colab.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 추가전략 2
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegressionCV

# ✅ 경로 함수
def get_path(filename):
    # Colab에서 파일을 업로드했거나 드라이브를 마운트했다면 해당 경로로 변경해야 합니다.
    # 예시: 구글 드라이브 마운트 후 'My Drive/data/' 폴더에 데이터가 있다면
    # return "/content/drive/MyDrive/data/" + filename
    # 현재 가정: Colab에 직접 파일을 업로드했고, 해당 파일들이 /content/open_1/ 에 있다고 가정
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
try:
    train_df = pd.read_csv(get_path("train.csv"))
    test_df = pd.read_csv(get_path("test.csv"))
    submission_df = pd.read_csv(get_path("sample_submission.csv")) # 변수명 통일
except FileNotFoundError:
    print("데이터셋 경로를 찾을 수 없습니다. Colab에 파일을 올바르게 업로드했는지 확인하거나 get_path 함수를 수정해주세요.")
    print("예: train.csv, test.csv, sample_submission.csv 파일이 /content/open_1/ 경로에 있는지 확인하세요.")
    exit() # 파일이 없으면 스크립트 종료

y = train_df["Cancer"]
X_raw = train_df.drop(columns=["ID", "Cancer"]) # 원본 X (전처리 전)
X_test_raw = test_df.drop(columns=["ID"]) # 원본 X_test (전처리 전)

# ✅ 전처리 전 원본 X_raw에서 범주형 컬럼 리스트 미리 추출 (전역에서 사용)
original_categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()

# ✅ 전처리 함수
def preprocess(df, encoders=None, is_train=True):
    df_processed = df.copy()
    encoders = encoders or {}

    for col in df_processed.columns:
        if col in original_categorical_cols: # 원본 범주형 컬럼에 대해서만 Label Encoding
            df_processed[col] = df_processed[col].astype(str)
            if is_train and col not in encoders:
                encoders[col] = LabelEncoder().fit(df_processed[col])
            # test 데이터에 train에 없는 새로운 범주가 나타날 경우를 대비
            df_processed[col] = df_processed[col].map(lambda s: encoders[col].transform([s])[0] if s in encoders[col].classes_ else -1)
        # elif df_processed[col].dtype == "object": # 새로운 FE 컬럼이 object 타입인 경우 (현재 코드에는 해당 안 됨)
        #     pass
        else: # 수치형 컬럼 (결측치 처리)
            imputer_key = f"imputer_{col}"
            if is_train:
                encoders[imputer_key] = SimpleImputer(strategy="mean").fit(df_processed[[col]])
            if imputer_key in encoders: # train에서 학습된 imputer가 있는 경우 사용
                df_processed[col] = encoders[imputer_key].transform(df_processed[[col]])
    return df_processed, encoders

# 데이터 전처리 적용
X_processed, encoders_fitted = preprocess(X_raw, is_train=True)
X_test_processed, _ = preprocess(X_test_raw, encoders=encoders_fitted, is_train=False) # encoders_fitted 사용

# ✅ 상호작용 변수 생성 함수
def add_interactions(df, cols):
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    inter_feats_array = poly.fit_transform(df[cols])

    # get_feature_names_out으로 생성될 컬럼 이름 가져오기
    feature_names = poly.get_feature_names_out(cols)

    # 상호작용 피처만 포함하는 DataFrame 생성
    # PolynomialFeatures가 (N)개의 원본 피처를 받으면, 첫 N개 컬럼 이름은 원본 피처 이름과 동일
    # 그 이후부터가 순수 상호작용 피처
    interaction_only_df = pd.DataFrame(inter_feats_array, columns=feature_names, index=df.index).iloc[:, len(cols):]

    return pd.concat([df, interaction_only_df], axis=1)

# Feature Interaction 적용
interaction_cols = X_processed.select_dtypes(include=np.number).columns.tolist()
X_inter = add_interactions(X_processed, interaction_cols)
X_test_inter = add_interactions(X_test_processed, interaction_cols)

# CatBoost를 위한 범주형 피처 인덱스 업데이트 (상호작용 피처 추가 후)
# X_inter (모든 전처리 및 상호작용 피처가 추가된 최종 DataFrame)에서
# original_categorical_cols에 해당하는 컬럼들의 현재 인덱스를 다시 찾습니다.
cat_features_indices_for_catboost_final = [
    X_inter.columns.get_loc(col) for col in original_categorical_cols if col in X_inter.columns
]


# ✅ 모델 정의
# CatBoost에 cat_features 명시
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42, n_jobs=-1, verbose=-1)
# Colab 무료 티어는 GPU가 항상 가용하지 않으므로 일단 CPU로 설정. 필요시 'GPU'로 변경
model_cat = CatBoostClassifier(verbose=0, random_state=42, cat_features=cat_features_indices_for_catboost_final, task_type='CPU')
meta_model = LogisticRegressionCV(cv=5, max_iter=1000, random_state=42, n_jobs=-1) # 메타 모델에도 random_state 및 n_jobs 추가

# ✅ 스태킹 모델 정의
stack_model = StackingClassifier(
    estimators=[('lgbm', model_lgbm), ('cat', model_cat)],
    final_estimator=meta_model,
    passthrough=True, # 기본 모델의 예측 외에 원본 특성도 메타 모델에 전달
    cv=5, # Stacking 내부의 교차 검증 폴드 수
    stack_method='predict_proba', # 기본 모델의 확률 예측을 메타 모델에 전달
    n_jobs=-1 # 병렬 처리
)

# ✅ OOF 예측 및 Threshold 최적화 (F1 Score 기반 최적화 전략)
print("Stacking 모델 OOF 예측 시작...")
# StackingClassifier의 cross_val_predict는 내부적으로 Stacking 로직을 수행하며 OOF 예측을 반환
oof_probs = cross_val_predict(stack_model, X_inter, y, method="predict_proba", cv=5)[:, 1]

thresholds = np.arange(0.1, 0.91, 0.01) # 0.91까지 포함하도록
metrics = []
for t in thresholds:
    # `pos_label=1`을 명시하여 Cancer=1 (악성) 클래스에 대한 지표를 계산
    preds_binary = (oof_probs > t).astype(int)
    f1 = f1_score(y, preds_binary, pos_label=1)
    prec = precision_score(y, preds_binary, pos_label=1, zero_division=0) # zero_division=0 추가
    rec = recall_score(y, preds_binary, pos_label=1, zero_division=0) # zero_division=0 추가
    metrics.append((t, f1, prec, rec))

# F1 Score가 가장 높은 임계값 찾기
best_thresh, best_f1, best_prec, best_rec = max(metrics, key=lambda x: x[1])
print(f"✅ Best Threshold: {best_thresh:.2f}, F1: {best_f1:.5f}, Precision: {best_prec:.5f}, Recall: {best_rec:.5f}")

# ✅ 최종 학습 및 예측
print("최종 Stacking 모델 학습 시작...")
stack_model.fit(X_inter, y) # 전체 훈련 데이터로 최종 Stacking 모델 학습
print("최종 Stacking 모델 예측 시작...")
test_probs = stack_model.predict_proba(X_test_inter)[:, 1] # 테스트 데이터에 대한 확률 예측

submission_df["Cancer"] = (test_probs > best_thresh).astype(int)
submission_df.to_csv("submission_stack_thresh_refined_colab.csv", index=False) # Colab용 파일명 변경
print("✅ 저장 완료: submission_stack_thresh_refined_colab.csv")

Stacking 모델 OOF 예측 시작...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the d

✅ Best Threshold: 0.18, F1: 0.48649, Precision: 0.51552, Recall: 0.46056
최종 Stacking 모델 학습 시작...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


최종 Stacking 모델 예측 시작...
✅ 저장 완료: submission_stack_thresh_refined_colab.csv


In [ ]:
from google.colab import files
files.download("submission_stack_thresh_refined_colab.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 추가전략 3
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    # Colab에서 파일을 업로드했거나 드라이브를 마운트했다면 해당 경로로 변경해야 합니다.
    # 예시: 구글 드라이브 마운트 후 'My Drive/data/' 폴더에 데이터가 있다면
    # return "/content/drive/MyDrive/data/" + filename
    # 현재 가정: Colab에 직접 파일을 업로드했고, 해당 파일들이 /content/open_1/ 에 있다고 가정
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
try:
    train_df = pd.read_csv(get_path("train.csv"))
    test_df = pd.read_csv(get_path("test.csv"))
    submission_df = pd.read_csv(get_path("sample_submission.csv"))
except FileNotFoundError:
    print("데이터셋 경로를 찾을 수 없습니다. Colab에 파일을 올바르게 업로드했는지 확인하거나 get_path 함수를 수정해주세요.")
    print("예: train.csv, test.csv, sample_submission.csv 파일이 /content/open_1/ 경로에 있는지 확인하세요.")
    exit() # 파일이 없으면 스크립트 종료

y = train_df["Cancer"]
X_raw = train_df.drop(columns=["ID", "Cancer"]) # 원본 X (전처리 전)
X_test_raw = test_df.drop(columns=["ID"]) # 원본 X_test (전처리 전)

# ✅ 전처리 전 원본 X_raw에서 범주형 컬럼 리스트 미리 추출 (전역에서 사용)
original_categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()

# ✅ 전처리 함수
def preprocess(df, encoders=None, is_train=True):
    df_processed = df.copy()
    encoders = encoders or {}

    for col in df_processed.columns:
        if col in original_categorical_cols: # 원본 범주형 컬럼에 대해서만 Label Encoding
            df_processed[col] = df_processed[col].astype(str)
            if is_train and col not in encoders:
                encoders[col] = LabelEncoder().fit(df_processed[col])
            # test 데이터에 train에 없는 새로운 범주가 나타날 경우를 대비
            df_processed[col] = df_processed[col].map(lambda s: encoders[col].transform([s])[0] if s in encoders[col].classes_ else -1)
        else: # 수치형 컬럼 (결측치 처리)
            imputer_key = f"imputer_{col}"
            if is_train:
                encoders[imputer_key] = SimpleImputer(strategy="mean").fit(df_processed[[col]])
            if imputer_key in encoders: # train에서 학습된 imputer가 있는 경우 사용
                df_processed[col] = encoders[imputer_key].transform(df_processed[[col]])
    return df_processed, encoders

# 데이터 전처리 적용
X_processed, encoders_fitted = preprocess(X_raw, is_train=True)
X_test_processed, _ = preprocess(X_test_raw, encoders=encoders_fitted, is_train=False) # encoders_fitted 사용

# ✅ 상호작용 변수 생성 함수
def add_interactions(df, cols):
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    inter_feats_array = poly.fit_transform(df[cols])

    feature_names = poly.get_feature_names_out(cols)

    # 상호작용 피처만 포함하는 DataFrame 생성
    interaction_only_df = pd.DataFrame(inter_feats_array, columns=feature_names, index=df.index).iloc[:, len(cols):]

    return pd.concat([df, interaction_only_df], axis=1)

# Feature Interaction 적용
interaction_cols = X_processed.select_dtypes(include=np.number).columns.tolist()
X_inter = add_interactions(X_processed, interaction_cols)
X_test_inter = add_interactions(X_test_processed, interaction_cols)

# CatBoost를 위한 범주형 피처 인덱스 업데이트 (상호작용 피처 추가 후)
# X_inter (모든 전처리 및 상호작용 피처가 추가된 최종 DataFrame)에서
# original_categorical_cols에 해당하는 컬럼들의 현재 인덱스를 다시 찾습니다.
cat_features_indices_for_catboost_final = [
    X_inter.columns.get_loc(col) for col in original_categorical_cols if col in X_inter.columns
]


# ✅ 모델 정의
# CatBoost에 cat_features 명시
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42, n_jobs=-1, verbose=-1)
# Colab 무료 티어는 GPU가 항상 가용하지 않으므로 일단 CPU로 설정. 필요시 'GPU'로 변경
model_cat = CatBoostClassifier(verbose=0, random_state=42, cat_features=cat_features_indices_for_catboost_final, task_type='CPU')
meta_model = LGBMClassifier(
    learning_rate=0.02,
    max_depth=3,
    num_leaves=8,
    colsample_bytree=0.7,
    subsample=0.8,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1, # n_jobs 추가
    verbose=-1 # verbose 추가
)

# ✅ 스태킹 모델 정의
# cross_val_predict와 StackingClassifier 모두 동일한 cv 객체를 사용하도록 통일
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stack_model = StackingClassifier(
    estimators=[('lgbm', model_lgbm), ('cat', model_cat)],
    final_estimator=meta_model,
    passthrough=True, # 기본 모델의 예측 외에 원본 특성도 메타 모델에 전달
    cv=cv_strategy, # Stacking 내부의 교차 검증 폴드 수
    stack_method='predict_proba', # 기본 모델의 확률 예측을 메타 모델에 전달
    n_jobs=-1 # 병렬 처리
)

# ✅ OOF 예측 및 Threshold 최적화
print("Stacking 모델 OOF 예측 시작...")
# cross_val_predict에도 동일한 cv 객체 사용
oof_preds = cross_val_predict(stack_model, X_inter, y, method="predict_proba", cv=cv_strategy)[:, 1]

thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"✅ Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 학습 및 테스트 예측
print("최종 Stacking 모델 학습 시작...")
stack_model.fit(X_inter, y) # 전체 훈련 데이터로 최종 Stacking 모델 학습
print("최종 Stacking 모델 예측 시작...")
test_preds = stack_model.predict_proba(X_test_inter)[:, 1] # 테스트 데이터에 대한 확률 예측

submission_df["Cancer"] = (test_preds > best_thresh).astype(int)
submission_df.to_csv("submission_stack_inter_tunedmeta_colab.csv", index=False) # Colab용 파일명 변경
print("✅ 저장 완료: submission_stack_inter_tunedmeta_colab.csv")

Stacking 모델 OOF 예측 시작...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ Best Threshold: 0.28, Best F1 Score: 0.48661
최종 Stacking 모델 학습 시작...
최종 Stacking 모델 예측 시작...
✅ 저장 완료: submission_stack_inter_tunedmeta_colab.csv


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from google.colab import files
files.download("submission_stack_inter_tunedmeta_colab.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # 전략 B-2: LGBMClassifier 메타 모델 기반 StackingClassifier 전체 코드
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold, cross_val_predict
# from sklearn.preprocessing import LabelEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import StackingClassifier
# from sklearn.metrics import f1_score
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

# # ✅ 경로 함수
# def get_path(filename):
#     return "/content/open_1/" + filename  # 환경에 맞게 경로 수정

# # ✅ 데이터 로딩
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# y = train["Cancer"]
# X = train.drop(columns=["ID", "Cancer"])
# X_test = test.drop(columns=["ID"])

# # ✅ 전처리 (라벨 인코딩 + 결측치)
# def preprocess(df, encoders=None):
#     df = df.copy()
#     encoders = encoders or {}
#     for col in df.columns:
#         if df[col].dtype == "object":
#             df[col] = df[col].astype(str)
#             if col not in encoders:
#                 encoders[col] = LabelEncoder().fit(df[col])
#             df[col] = encoders[col].transform(df[col])
#         else:
#             df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
#     return df, encoders

# X, encoders = preprocess(X)
# X_test, _ = preprocess(X_test, encoders)

# # ✅ 기본 모델
# model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
# model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
# model_cat = CatBoostClassifier(verbose=0, random_state=42)

# # ✅ 메타 모델 (LGBMClassifier로 변경)
# meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# # ✅ 스태킹 모델 정의
# base_models = [
#     ('lgbm', model_lgbm),
#     ('xgb', model_xgb),
#     ('cat', model_cat)
# ]

# stack_model = StackingClassifier(
#     estimators=base_models,
#     final_estimator=meta_model,
#     cv=5,
#     stack_method="predict_proba",
#     n_jobs=-1
# )

# # ✅ Threshold 최적화
# oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
# thresholds = np.arange(0.1, 0.9, 0.01)
# f1s = [f1_score(y, oof_preds > t) for t in thresholds]
# best_thresh = thresholds[np.argmax(f1s)]
# print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# # ✅ 최종 학습 및 예측
# stack_model.fit(X, y)
# test_preds = stack_model.predict_proba(X_test)[:, 1]
# submission["Cancer"] = (test_preds > best_thresh).astype(int)
# submission.to_csv("submission_stacking_lgb_meta.csv", index=False)

[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001934 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001956 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001926 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Threshold: 0.26, Best F1 Score: 0.48624


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from google.colab import files
files.download("submission_stacking_lgb_meta.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# feature 변수 확인해보기
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer # 기존 baseline과 동일하게 mean 사용
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename  # 환경에 맞게 경로 수정

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 및 Feature Engineering 함수 통합
def preprocess_and_engineer_features(df, encoders=None, is_train=True):
    df = df.copy()
    encoders = encoders or {}

    # --- 1. 기존 전처리 (Label Encoding + Mean Imputation) ---
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            # SimpleImputer는 2D array를 기대하므로 df[[col]] 사용
            # Baseline과 동일하게 mean strategy 사용
            # 훈련 데이터에서 학습하고, 테스트 데이터에서는 학습된 Imputer 사용
            imputer_key = f"imputer_{col}"
            if is_train:
                encoders[imputer_key] = SimpleImputer(strategy="mean").fit(df[[col]])
            if imputer_key in encoders:
                df[col] = encoders[imputer_key].transform(df[[col]])
            else: # If imputer not fit for this column (e.g., no NaNs in train for this col)
                # This case should ideally not happen if all numerical columns are consistently processed
                pass # Or raise an error, or fit here if it's new numerical col in test only (unlikely)


    # --- 2. 새로운 Feature Engineering ---

    # 로그 변환 (Nodule_Size 및 호르몬 결과)
    for col in ['Nodule_Size', 'TSH_Result', 'T4_Result', 'T3_Result']:
        if col in df.columns:
            # 0 또는 음수 값 처리를 위해 np.log1p 사용 (log(x+1))
            # 무한대나 NaN 값이 있을 수 있으므로 미리 처리
            df[f'{col}_log'] = np.log1p(df[col].replace([np.inf, -np.inf], np.nan).fillna(0))

    # 호르몬 비율 변수 (0으로 나누기 방지)
    if 'TSH_Result' in df.columns and 'T4_Result' in df.columns:
        df['TSH_T4_Ratio'] = df['TSH_Result'] / (df['T4_Result'] + 1e-6)
    if 'T3_Result' in df.columns and 'T4_Result' in df.columns:
        df['T3_T4_Ratio'] = df['T3_Result'] / (df['T4_Result'] + 1e-6)

    # 연령 및 결절 크기 상호작용 변수
    if 'Age' in df.columns and 'Nodule_Size' in df.columns:
        df['Age_Nodule_Interaction'] = df['Age'] * df['Nodule_Size']
        df['Age_Nodule_Ratio'] = df['Age'] / (df['Nodule_Size'] + 1e-6)

    # 위험 요인 개수 (LabelEncoder로 변환된 값들이 0 또는 1 (False/True)로 잘 매핑되었다고 가정)
    # 실제 값은 0과 1이 아닌 정수형으로 변환될 수 있으므로, 해당 컬럼들을 직접 선택해야 함
    # 명시적으로 이진 변수로 가정하고 합산
    risk_factors_for_sum = [
        'Family_Background', 'Radiation_History', 'Iodine_Deficiency',
        'Smoke', 'Weight_Risk', 'Diabetes'
    ]
    # 이 컬럼들이 LabelEncoder에 의해 0 또는 1로 변환되었다고 가정
    # (LabelEncoder는 알파벳 순서에 따라 0, 1, 2... 로 매핑하므로 주의 필요)
    # 실제 값 확인 후 필요하면 수동 매핑 로직 추가

    # 안전하게, LabelEncoder를 통과한 'object' 타입 컬럼 중 위험 요인으로 판단되는 컬럼들만 선택하여 합산
    # 'Weight_Risk'는 'Low', 'Normal', 'High' 같은 값일 수 있으므로 이진 합산에 부적합할 수 있음.
    # 여기서는 예시로 다음 컬럼들을 가정하고 합산합니다.
    binary_risk_factors = [col for col in risk_factors_for_sum if col in df.columns] # 존재하는 컬럼만 선택
    if binary_risk_factors:
        df['Num_Risk_Factors'] = df[binary_risk_factors].sum(axis=1)


    return df, encoders

# 데이터 전처리 및 Feature Engineering 적용
# train 데이터로 encoders (LabelEncoder, SimpleImputer) 학습
X, encoders = preprocess_and_engineer_features(X, is_train=True)
# test 데이터에 학습된 encoders 적용
X_test, _ = preprocess_and_engineer_features(X_test, encoders, is_train=False)

# CatBoost가 범주형 피처를 올바르게 인식하도록 컬럼 이름 기반으로 인덱스 찾기
# 주의: LabelEncoder로 변환된 컬럼들 중 실제로 범주형으로 처리되어야 하는 것만 명시
# CatBoost는 기본적으로 정수형 피처도 범주형으로 간주할 수 있으므로,
# 모든 LabelEncoded 컬럼을 cat_features로 넣어주는 것이 안전합니다.
cat_features_indices_for_catboost = [
    i for i, col in enumerate(X.columns)
    if col in ['Gender', 'Country', 'Race', 'Family_Background', 'Radiation_History',
               'Iodine_Deficiency', 'Smoke', 'Weight_Risk', 'Diabetes'] # 원본 범주형 컬럼들
]


# ✅ 기본 모델
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
# CatBoost는 명시적으로 cat_features를 지정할 수 있도록 수정
model_cat = CatBoostClassifier(verbose=0, random_state=42,
                               cat_features=cat_features_indices_for_catboost)

# ✅ 메타 모델 (LGBMClassifier로 변경)
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# ✅ 스태킹 모델 정의
base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 학습 및 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_lgb_meta_fe.csv", index=False) # 파일명 변경

print("Submission file created successfully: submission_stacking_lgb_meta_fe.csv")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001916 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011500 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003333 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Threshold: 0.25, Best F1 Score: 0.48639
[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003914 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from google.colab import files
files.download("submission_stacking_lgb_meta_fe.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
import optuna # Optuna 라이브러리 임포트

# ✅ 경로 함수 (Colab 환경 경로. VS Code 로컬 환경에서는 "D:/traffic-project/" 등으로 수정)
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train_df = pd.read_csv(get_path("train.csv"))
test_df = pd.read_csv(get_path("test.csv"))
submission_df = pd.read_csv(get_path("sample_submission.csv"))

y = train_df["Cancer"]
X_raw = train_df.drop(columns=["ID", "Cancer"]) # 전처리 전 원본 X
X_test_raw = test_df.drop(columns=["ID"]) # 전처리 전 원본 X_test

# ✅ 전처리 및 Feature Engineering 함수
def preprocess_and_engineer_features(df, encoders=None, is_train=True):
    df = df.copy()
    encoders = encoders or {}

    # 1. Label Encoding 및 Mean Imputation
    # 원본 범주형 컬럼 리스트 (LabelEncoder 적용 대상)
    original_categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()

    for col in df.columns:
        if col in original_categorical_cols: # 원본 범주형 컬럼만 Label Encoding
            df[col] = df[col].astype(str) # ensure string type
            if is_train and col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            # 테스트 데이터에 훈련 데이터에 없는 범주가 나타날 경우 KeyError 방지
            # Unseen labels are mapped to -1, which models should ideally handle
            df[col] = df[col].map(lambda s: encoders[col].transform([s])[0] if s in encoders[col].classes_ else -1)
            # -1은 모델에 따라 다르게 해석될 수 있으므로, 실제 프로젝트에서는 다른 전략 (e.g. OHE + handle_unknown='ignore') 권장

        elif df[col].dtype == "object": # Label Encoding 대상이 아닌데 object 타입인 경우 (새로운 FE 컬럼 등)
            # 이 코드는 기본적으로 모든 'object' 컬럼을 Label Encoding하려고 시도함
            # 새로운 FE 컬럼이 object 타입으로 생성되지 않는다고 가정
            pass # 또는 float 등으로 강제 변환

        else: # 수치형 컬럼
            imputer_key = f"imputer_{col}"
            if is_train:
                encoders[imputer_key] = SimpleImputer(strategy="mean").fit(df[[col]])
            if imputer_key in encoders:
                df[col] = encoders[imputer_key].transform(df[[col]])

    # --- 2. 새로운 Feature Engineering ---
    # 로그 변환 (Nodule_Size 및 호르몬 결과)
    for col in ['Nodule_Size', 'TSH_Result', 'T4_Result', 'T3_Result']:
        if col in df.columns:
            df[f'{col}_log'] = np.log1p(df[col].replace([np.inf, -np.inf], np.nan).fillna(0))

    # 호르몬 비율 변수 (0으로 나누기 방지)
    if 'TSH_Result' in df.columns and 'T4_Result' in df.columns:
        df['TSH_T4_Ratio'] = df['TSH_Result'] / (df['T4_Result'] + 1e-6)
    if 'T3_Result' in df.columns and 'T4_Result' in df.columns:
        df['T3_T4_Ratio'] = df['T3_Result'] / (df['T4_Result'] + 1e-6)

    # 연령 및 결절 크기 상호작용 변수
    if 'Age' in df.columns and 'Nodule_Size' in df.columns:
        df['Age_Nodule_Interaction'] = df['Age'] * df['Nodule_Size']
        df['Age_Nodule_Ratio'] = df['Age'] / (df['Nodule_Size'] + 1e-6)

    # 위험 요인 개수 (LabelEncoder로 변환된 값들이 0 또는 1 (False/True)로 잘 매핑되었다고 가정)
    # 실제 값은 LabelEncoder에 의해 0, 1, 2... 로 매핑될 수 있으므로,
    # 'Yes'/'No' 형태의 이진 컬럼만 합산하도록 명시적 조정이 필요할 수 있음.
    # 여기서는 원본 이진 성격의 컬럼만 가정합니다.
    binary_risk_factors_cols = [
        'Family_Background', 'Radiation_History', 'Iodine_Deficiency',
        'Smoke', 'Diabetes' # Weight_Risk는 이진이 아닐 수 있어 일단 제외
    ]
    # LabelEncoder에 의해 0 또는 1로 잘 매핑되었다고 가정한 컬럼들만 합산
    # 'Weight_Risk'는 'Low', 'Normal', 'High' 같은 값일 수 있어 합산에서 제외.
    # 필요시 Weight_Risk도 이진화 (예: High만 1, 나머지는 0) 후 합산 가능.
    actual_risk_cols = [col for col in binary_risk_factors_cols if col in df.columns]
    if actual_risk_cols:
        df['Num_Risk_Factors'] = df[actual_risk_cols].sum(axis=1)

    return df, encoders

# 데이터 전처리 및 Feature Engineering 적용 (Optuna Objective 밖에서 한 번만 수행)
X_processed, encoders_fitted = preprocess_and_engineer_features(X_raw, is_train=True)

# CatBoost를 위한 범주형 피처 인덱스 (전처리 후 업데이트 필요)
# 원본 범주형 컬럼들만 cat_features로 지정
original_categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()
cat_features_indices_for_catboost = [
    X_processed.columns.get_loc(col) for col in original_categorical_cols if col in X_processed.columns
]


# ✅ Optuna Objective 함수 정의
def objective(trial):
    # --- 기본 모델 하이퍼파라미터 정의 ---
    # LGBMClassifier
    lgbm_params = {
        'objective': 'binary',
        'metric': 'binary_f1',
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 500, 3000, step=100),
        'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 20, 100, step=5),
        'max_depth': trial.suggest_int('lgbm_max_depth', 5, 15),
        'colsample_bytree': trial.suggest_float('lgbm_colsample_bytree', 0.5, 0.9),
        'subsample': trial.suggest_float('lgbm_subsample', 0.5, 0.9),
        'reg_alpha': trial.suggest_float('lgbm_reg_alpha', 0.0, 0.5),
        'reg_lambda': trial.suggest_float('lgbm_reg_lambda', 0.0, 0.5),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1,
    }

    # XGBClassifier
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'n_estimators': trial.suggest_int('xgb_n_estimators', 500, 3000, step=100),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('xgb_max_depth', 5, 15),
        'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.5, 0.9),
        'subsample': trial.suggest_float('xgb_subsample', 0.5, 0.9),
        'gamma': trial.suggest_float('xgb_gamma', 0.0, 0.5),
        'reg_alpha': trial.suggest_float('xgb_reg_alpha', 0.0, 0.5),
        'reg_lambda': trial.suggest_float('xgb_reg_lambda', 0.0, 0.5),
        'random_state': 42,
        'n_jobs': -1,
        'tree_method': 'hist',
    }

    # CatBoostClassifier
    cat_params = {
        'objective': 'Logloss',
        'iterations': trial.suggest_int('cat_iterations', 500, 3000, step=100),
        'learning_rate': trial.suggest_float('cat_learning_rate', 0.01, 0.1, log=True),
        'depth': trial.suggest_int('cat_depth', 5, 10),
        'l2_leaf_reg': trial.suggest_float('cat_l2_leaf_reg', 1.0, 10.0),
        'random_seed': 42,
        'verbose': 0,
        'cat_features': cat_features_indices_for_catboost, # 범주형 피처 지정
        'od_type': 'Iter',
        'od_wait': 50, # early stopping rounds
    }

    # --- 메타 모델 하이퍼파라미터 정의 ---
    meta_lgbm_params = {
        'objective': 'binary',
        'metric': 'binary_f1',
        'n_estimators': trial.suggest_int('meta_lgbm_n_estimators', 100, 1000, step=50),
        'learning_rate': trial.suggest_float('meta_lgbm_learning_rate', 0.01, 0.05, log=True),
        'num_leaves': trial.suggest_int('meta_lgbm_num_leaves', 10, 30, step=5),
        'max_depth': trial.suggest_int('meta_lgbm_max_depth', 3, 7),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1,
    }

    model_lgbm = LGBMClassifier(**lgbm_params)
    model_xgb = XGBClassifier(**xgb_params)
    model_cat = CatBoostClassifier(**cat_params)
    meta_model = LGBMClassifier(**meta_lgbm_params)

    base_models = [
        ('lgbm', model_lgbm),
        ('xgb', model_xgb),
        ('cat', model_cat)
    ]

    stack_model = StackingClassifier(
        estimators=base_models,
        final_estimator=meta_model,
        cv=5, # 5-Fold Cross Validation
        stack_method="predict_proba",
        n_jobs=-1
    )

    # 교차 검증을 통한 F1 Score 계산 (OOF 예측)
    # StackingClassifier가 내부적으로 cross_val_predict를 수행하여 OOF 예측을 반환
    oof_preds = cross_val_predict(stack_model, X_processed, y, method="predict_proba", cv=5)[:, 1]

    # 최적의 Threshold를 찾아 F1 Score 계산
    thresholds = np.arange(0.1, 0.9, 0.01)
    f1s = [f1_score(y, oof_preds > t, average='binary') for t in thresholds]

    if not f1s: # f1s 리스트가 비어있을 경우 (예외 처리)
        return 0.0

    return max(f1s) # 최대 F1 Score 반환

# --- Optuna Study 생성 및 최적화 실행 ---
# sampler=optuna.samplers.TPESampler(seed=42)를 추가하여 재현성 확보
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))

# n_trials는 컴퓨팅 자원과 시간 제약에 맞춰 충분히 크게 설정
# 여기서는 예시로 50회 설정 (더 높게 100~200 이상 추천)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\nNumber of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print(f"  Value (Best OOF F1 Score from Optuna tuning): {trial.value:.5f}")
print("  Best Parameters: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# --- 최적 하이퍼파라미터로 최종 모델 학습 및 예측 ---
best_params = study.best_params

# 기본 모델 파라미터 분리
lgbm_best_params = {k.replace('lgbm_', ''): v for k, v in best_params.items() if k.startswith('lgbm_')}
xgb_best_params = {k.replace('xgb_', ''): v for k, v in best_params.items() if k.startswith('xgb_')}
cat_best_params = {k.replace('cat_', ''): v for k, v in best_params.items() if k.startswith('cat_')}
meta_lgbm_best_params = {k.replace('meta_lgbm_', ''): v for k, v in best_params.items() if k.startswith('meta_lgbm_')}

# CatBoost의 cat_features는 튜닝 대상이 아니므로 고정된 값 사용
if 'cat_features' not in cat_best_params:
    cat_best_params['cat_features'] = cat_features_indices_for_catboost

# 모델 인스턴스 생성 (재현성을 위해 random_state, verbose 등 고정 파라미터도 추가)
model_lgbm_final = LGBMClassifier(**lgbm_best_params, objective='binary', metric='binary_f1', random_state=42, n_jobs=-1, verbose=-1)
model_xgb_final = XGBClassifier(**xgb_best_params, objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, random_state=42, n_jobs=-1, tree_method='hist')
model_cat_final = CatBoostClassifier(**cat_best_params, objective='Logloss', random_seed=42, verbose=0)

meta_model_final = LGBMClassifier(**meta_lgbm_best_params, objective='binary', metric='binary_f1', random_state=42, n_jobs=-1, verbose=-1)

base_models_final = [
    ('lgbm', model_lgbm_final),
    ('xgb', model_xgb_final),
    ('cat', model_cat_final)
]

stack_model_final = StackingClassifier(
    estimators=base_models_final,
    final_estimator=meta_model_final,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# 최종 모델 학습 (전체 데이터 사용)
stack_model_final.fit(X_processed, y)

# 테스트 데이터 전처리 (X_test_raw에 대해 encoders_fitted 사용)
X_test_processed, _ = preprocess_and_engineer_features(X_test_raw, encoders_fitted, is_train=False)

# 최종 예측
test_preds = stack_model_final.predict_proba(X_test_processed)[:, 1]

# 최종 학습된 모델의 OOF 예측을 다시 생성하여 F1 Score 계산 및 Threshold 탐색
# (이것이 튜닝 과정에서 얻은 trial.value보다 더 정확한 최종 성능 지표가 됨)
oof_preds_final_model = cross_val_predict(stack_model_final, X_processed, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds_final_model > t) for t in thresholds]
best_thresh_final = thresholds[np.argmax(f1s)]

print(f"\nFinal Model - Best Threshold for Submission: {best_thresh_final:.2f}, Final OOF F1 Score: {max(f1s):.5f}")

submission_df["Cancer"] = (test_preds > best_thresh_final).astype(int)
submission_df.to_csv("submission_stacking_lgbm_optuna_fe.csv", index=False)

print("Submission file created successfully: submission_stacking_lgbm_optuna_fe.csv")

[I 2025-06-26 11:45:27,451] A new study created in memory with name: no-name-bfc32daa-b5a1-4452-bfdc-70a567961c64


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[I 2025-06-26 12:37:51,523] Trial 0 finished with value: 0.48562332609025216 and parameters: {'lgbm_n_estimators': 1400, 'lgbm_learning_rate': 0.08927180304353628, 'lgbm_num_leaves': 80, 'lgbm_max_depth': 11, 'lgbm_colsample_bytree': 0.5624074561769746, 'lgbm_subsample': 0.562397808134481, 'lgbm_reg_alpha': 0.02904180608409973, 'lgbm_reg_lambda': 0.4330880728874676, 'xgb_n_estimators': 2000, 'xgb_learning_rate': 0.051059032093947576, 'xgb_max_depth': 5, 'xgb_colsample_bytree': 0.8879639408647977, 'xgb_subsample': 0.8329770563201687, 'xgb_gamma': 0.10616955533913808, 'xgb_reg_alpha': 0.09091248360355031, 'xgb_reg_lambda': 0.09170225492671691, 'cat_iterations': 1200, 'cat_learning_rate': 0.03347776308515933, 'cat_depth': 7, 'cat_l2_leaf_reg': 3.6210622617823773, 'meta_lgbm_n_estimators': 650, 'meta_lgbm_learning_rate': 0.012517051076140219, 'meta_lgbm_num_leaves': 15, 'meta_lgbm_max_depth': 4}. Best is trial 0 with value: 0.48562332609025216.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[I 2025-06-26 13:12:18,001] Trial 1 finished with value: 0.48515151515151517 and parameters: {'lgbm_n_estimators': 1600, 'lgbm_learning_rate': 0.06097839109531514, 'lgbm_num_leaves': 35, 'lgbm_max_depth': 10, 'lgbm_colsample_bytree': 0.736965827544817, 'lgbm_subsample': 0.5185801650879991, 'lgbm_reg_alpha': 0.3037724259507192, 'lgbm_reg_lambda': 0.08526206184364576, 'xgb_n_estimators': 600, 'xgb_learning_rate': 0.08889667907018929, 'xgb_max_depth': 15, 'xgb_colsample_bytree': 0.8233589392465845, 'xgb_subsample': 0.6218455076693483, 'xgb_gamma': 0.048836057003191935, 'xgb_reg_alpha': 0.34211651325607845, 'xgb_reg_lambda': 0.22007624686980065, 'cat_iterations': 800, 'cat_learning_rate': 0.03127353036780371, 'cat_depth': 5, 'cat_l2_leaf_reg': 9.18388361870904, 'meta_lgbm_n_estimators': 300, 'meta_lgbm_learning_rate': 0.029045790726652743, 'meta_lgbm_num_leaves': 15, 'meta_lgbm_max_depth': 5}. Best is trial 0 with value: 0.48562332609025216.
